In [1]:
# We need to join the upper directory in order to access the local modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import itertools
import json
import logging

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

In [3]:
import numpy as np
import pandas as pd
import scipy

In [4]:
import pytdoa.geodesy.geodesy as geodesy
import pytdoa.ltess.ltess as ltess
import pytdoa.mlat.nlls as nlls
from pytdoa.pytdoa import correct_fo
from pytdoa.spec_load import spec_load
import pytdoa.tdoa.tdoa as tdoa

In [5]:
cfg_file = "../.config/config.json"
with open(cfg_file) as f:
    config = json.load(f)

In [6]:
# Load configuration
# Unknown transmitter
fUS_MHz = config["transmitters"]["unknown"]["freq"]
fUS = fUS_MHz * 1e6

# Reference transmitter
fRS_MHz = config["transmitters"]["reference"]["freq"]
fRS = fRS_MHz * 1e6
rs_lat = config["transmitters"]["reference"]["coord"][0]
rs_lon = config["transmitters"]["reference"]["coord"][1]
rs_alt = config["transmitters"]["reference"]["height"]
rs_llh = np.array([rs_lat, rs_lon, rs_alt]).reshape(1, 3)

# Sensor configurations
sr_tdoa = config["config"]["sample_rate"]
sr_ltess = config["config"]["sample_rate_ltess"]
sensors = pd.DataFrame(config["sensors"])
sensors[["latitude", "longitude"]] = sensors.coordinates.to_list()
sensors = sensors.drop(["coordinates"], axis=1)
directory = config["config"]["folder"]
filenum = config["config"]["filenum"]
NUM_SENSORS = len(sensors)

# TDOA estimation
corr_type = config["config"].get("corr_type", "dphase")
interpol = config["config"].get("interpol", 1)
bw_rs = config["transmitters"]["reference"].get("bw", sr_tdoa)
bw_us = config["transmitters"]["unknown"].get("bw", sr_tdoa)

# Design the filter taps for all chunks
taps_rs, taps_us = None, None
if bw_rs < sr_tdoa:
    taps_rs = tdoa.design_filt(bw_rs, sr_tdoa)
    logger.info(f"Filter for reference signal of bandwidth {bw_rs:.2f} created")

if bw_us < sr_tdoa:
    taps_us = tdoa.design_filt(bw_us, sr_tdoa)
    logger.info(f"Filter for targe signal of bandwidth {bw_us:.2f} created")

# Correct the drift of the RTL-SDRs (requires knowing the drift in PPM)
correct = config["config"]["correct"]

# Return the accurate position from NLLS
method = config["config"].get("method", "linear")

In [94]:
selected_list = ["madrid_city_1", "mad_princesa", "chamberi", "rack_1"]
sensorsf = sensors.loc[sensors['name'].isin(selected_list)].copy()

In [57]:
# Hack to reorder indices
sensorsf.index = [1,0,2]
sensorsf = sensorsf.sort_index()

In [58]:
sensorsf

,name,height,serial,id,latitude,longitude
0,chamberi,600,202481591991736,1,40.43450,-3.7134
1,mad_princesa,600,202481589193614,6,40.43515,-3.6742
2,madrid_city_1,600,202481596393530,2,40.39970,-3.7067


In [21]:
def get_samples_per_sensor(sensors):
    samples = {}
    for (i, sensor) in sensors.iterrows():
        # Computing the distance to the Ref tX
        sensor_llh = np.array(
            [sensor["latitude"], sensor["longitude"], sensor["height"]]
        ).reshape(1, 3)
        dist = geodesy.dist3fromllh(rs_llh, sensor_llh)
        sensors.at[i, "ref_dist"] = dist

        # Compute ECEF coordinates per sensor
        ecef = geodesy.llh2ecef(sensor_llh).squeeze()
        sensors.at[i, "x"] = ecef[0]
        sensors.at[i, "y"] = ecef[1]
        sensors.at[i, "z"] = ecef[2]

        # Loading IQ data
        sname = sensor["name"]
        fname_tdoa = f"{directory}/{sname}/E{filenum}-{int(fRS_MHz)}_{int(fUS_MHz)}-localization.dat"
        fname_ltess = f"{directory}/{sname}/E{filenum}-ltess.dat"

        tdoa_iq = spec_load(fname_tdoa)
        ltess_iq = spec_load(fname_ltess)

        if correct:
            # Estimate Clock drift using the LTESS-Track tool
            (PPM, _, _) = ltess.ltess(ltess_iq, resample_factor=60)
            # Clock correction
            samples[sname] = correct_fo(tdoa_iq, PPM, fRS, fUS, samplingRate=sr_tdoa)
        else:
            samples[sname] = tdoa_iq

    return (samples)

In [95]:
samples = get_samples_per_sensor(sensorsf)

In [60]:
def compute_tdoa_combinations(sensors, samples, combinations):
    combination_list = np.empty((0, 2), dtype=int)
    tdoa_list = np.empty(0)

    for combination in combinations:
        i, j = combination[0], combination[1]

        combination_list = np.vstack(
            (combination_list, [combination[0], combination[1]])
        )

        name_i = sensors.iloc[i]["name"]
        name_j = sensors.iloc[j]["name"]
        rx_diff = sensors.iloc[i]["ref_dist"] - sensors.iloc[j]["ref_dist"]

        tdoa_ij = tdoa.tdoa(
            samples[name_i],
            samples[name_j],
            rx_diff,
            interpol=interpol,
            corr_type=corr_type,
            taps_rs=taps_rs,
            taps_us=taps_us,
        )
        tdoa_list = np.append(tdoa_list, [tdoa_ij["tdoa_m_i"]])

    return (combination_list, tdoa_list)

In [96]:
# Get combinations and compute TDOAs per pair
combinations = itertools.combinations(np.arange(len(sensorsf)), 2)
(combination_list, tdoa_list) = compute_tdoa_combinations(sensorsf, samples, combinations)

In [97]:
sensors_llh = sensorsf[["latitude", "longitude", "height"]].to_numpy().reshape(-1,3)
sensors_ecef = geodesy.llh2ecef(sensors[["latitude", "longitude", "height"]].to_numpy())
sensors_mean = np.mean(sensors_ecef, axis=0)

optimfun = lambda X: nlls.nlls(X, sensors_ecef - sensors_mean, tdoa_list, combination_list)

In [98]:
res = scipy.optimize.minimize(optimfun, sensors_mean, method="BFGS")

In [99]:
res_ecef = (res.x + sensors_mean).reshape(1,3)
res_llh = geodesy.ecef2llh(res_ecef)

In [100]:
res_llh[0,0:2]

array([40.2772289 , -3.75514172])

In [101]:
ltrange = 0.5
lnrange = 0.5
step=0.01
Xs = [40.42053395579597, -3.664194425799879]
lat_range = np.linspace(Xs[0]-ltrange, Xs[0]+ltrange, round((2*ltrange)/step))
lon_range = np.linspace(Xs[1]-lnrange, Xs[1]+lnrange, round((2*lnrange)/step))

In [102]:
corr_result = np.empty((lat_range.shape[0], lon_range.shape[0]))

In [65]:
for (i, latitude) in enumerate(lat_range):
    for (j,longitude) in enumerate(lon_range):
        corr_result[i,j] = optimfun([latitude,longitude])

In [66]:
corr_result_db = 10*np.log10(corr_result)

In [67]:
ind = np.unravel_index(np.argmin(corr_result_db, axis=None), corr_result_db.shape)

In [68]:
print(f"{lat_range[ind[0]]}, {lon_range[ind[1]]}")
print(corr_result_db[ind])

print(f"{Xs[0]}, {Xs[1]}")
print(10*np.log10(optimfun([40.42053395579597, -3.664194425799879])))

40.647806683068694, -4.164194425799879
49.35376305675244
40.42053395579597, -3.664194425799879
76.10610305414664


In [69]:
corr_result_db = 10 * np.log10(corr_result)

In [70]:
from mpl_toolkits import mplot3d

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib qt
sns.set_style("white")
sns.set_context("notebook")

In [71]:

plt.figure(figsize=(20, 20), dpi=300)

(LT, LN) = np.meshgrid(lat_range, lon_range)

ax = plt.axes(projection='3d')
ax.plot_surface(LT, LN, corr_result_db, rstride=1, cstride=1,
                cmap='viridis', edgecolor='none')
ax.scatter(lat_range[ind[0]], lon_range[ind[1]], corr_result_db[ind], marker='x')
ax.scatter(40.4204657077291, -3.6641990667656934, 10*np.log10(optimfun([40.4204657077291, -3.6641990667656934])), marker='x')
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
plt.show()